## Markerless Mice Tracking for Social Experiments
This is an implementation of the pipeline described in the paper. 
* Input of the pipeline can be a video or a directory containing frames in sequence. 
* Output of the workflow are two csv files storing coordinates of snout and tailbase of corresponding to two mice: *features_mouse1_ensemble.csv* and *features_mouse2_ensemble.csv*
    

This pipeline used a pretrained Mask-RCNN model and a pretrained DeepLabCut model provided in subfolders mrcnn_models and dlc_models. However, Mask-RCNN and DeepLabCut models should be retrained on new data if the settings of new videos are different with our setting in the paper. The code to train those models can be found in two Jupyter Notebooks as follows:
* *deeplabcut_training.ipynb*
* *mrcnn_training.ipynb*


In [ ]:
import os
import sys

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  # To find local version of the library

from mouse.utils import labelmejson_to_png
from mouse.utils import video2frames, background_subtraction
from mouse.utils import correct_segmentation_errors, mouse_mrcnn_segmentation, check_mrcnn_model_path
from mouse.utils import tracking_inference, mask_based_detection, mice_separation
from mouse.utils import deeplabcut_detection, ensemble_features


### Path to cage background which is normally imaged before the experiment

In [ ]:
background_dir='..\\videos\\BG1.jpg'                                       #----------------update your cage background

### Path the a new video or frames
The input of the pipeline can be a video or a directory containing a sequence of frames. 

In [ ]:
# Paths to a new video 
tracking_video_dir = "..\\videos\\video1.avi"                            #----------------update if input is a video

# Extracting the video into frames 
print('The video will be extracted into frames stored in {}'.format(os.path.splitext(tracking_video_dir)[0] + '\\images'))
video2frames(tracking_video_dir)  

frames_dir = os.path.join(os.path.splitext(tracking_video_dir)[0], 'images')

In [ ]:
# Path to frames
#frames_dir = '..\\videos\\video1\images'                                 #-------------------update if input are frames

### Step 1: Conventional foreground detection to segment mice

In [ ]:
components = background_subtraction(frames_dir, background_dir)

### Step 2: Mask-RCNN to segment mice in the failed frames in step 1
Update MODEL_PATH for a specific model you want to use. If MODEL_PATH == None, the weights of your last trained model will be loaded

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "mrcnn_models")

# Directory to load weights of a model. If MRCNN_MODEL_PATH=None, the latest weights in MODEL_DIR will be loaded
MRCNN_MODEL_PATH = "..\\mrcnn_models\\mask_rcnn_mouse_0025.h5"  # None

if MRCNN_MODEL_PATH != None:
    if not os.path.exists(MRCNN_MODEL_PATH):
        print("Please read mrcnn_models/README.md to download our trained model")
    
    else:
        components = mouse_mrcnn_segmentation(components, frames_dir, background_dir, model_dir=MODEL_DIR, model_path=MRCNN_MODEL_PATH)
        
elif MRCNN_MODEL_PATH == None:
    if check_mrcnn_model_path(MODEL_DIR):
        print("The latest trained model will be loaded")
        components = mouse_mrcnn_segmentation(components, frames_dir, background_dir, model_dir=MODEL_DIR, model_path=MRCNN_MODEL_PATH)
    else:
        print("Could not find model directory under {}".format(MODEL_DIR))
        print("Please follow the pipeline in mrcnn_training.ipynb to train your own model, then run this step again.")

### Step 3: Suggesting intervention
Althought we did not intervene the Mask-RCNN results in the paper, it is recommended to fix very 3 consecutive failed frames. 
Skip step 3 if you don't want to correct the mistakes.

To correct segmentation errors, in Labelme GUI open *fix_dir* and annotate both mice as a label of *mouse*

In [ ]:
fix_dir = os.path.join(os.path.dirname(frames_dir), 'suggested_fix')
os.mkdir(fix_dir)

suggesting_correction = correct_segmentation_errors(components, fix_dir, frames_dir)

if suggesting_correction==0:
    print('There are no frames for correcting')
else:
    print('{} frames in the directory {} are recommended to be corrected using Labelme'.format(suggesting_correction, fix_dir))
    ! labelme

In [ ]:
# convert fix_frames back to FG and update components
output_dir = os.path.join(os.path.dirname(frames_dir),'FG')        
labelmejson_to_png(fix_dir, output_dir)      

### Step 4: Tracking inference

In [ ]:
fg_dir = os.path.join(os.path.dirname(frames_dir),'FG')
tracking_inference(fg_dir, components)

### Step 5:  Mask-based detection

In [ ]:
tracking_dir = os.path.join(os.path.dirname(frames_dir),'tracking')
features_mouse1_md, features_mouse2_md = mask_based_detection(tracking_dir, components)

### Step 6: Deeplabcut detection
Update config_dir to load your own trained model.

In [ ]:
# Separating the video into 2 videos corresponding to two animals.
mice_separation(tracking_dir, frames_dir, background_dir)   

In [ ]:
# Detect the key points by Deeplabcut model
config_dir = '../dlc_models/dlc_mice_tracking/config.yaml'              #------Update this for your own model

if os.path.exists(config_dir):
    video_dir = [os.path.join(os.path.dirname(tracking_dir), 'mouse1.avi'),
                 os.path.join(os.path.dirname(tracking_dir), 'mouse2.avi')]

    features_mouse1_dlc, features_mouse2_dlc = deeplabcut_detection(config_dir, video_dir)
else:
    print("Please read dlc_models/README.md to download our trained model and run this step again")
    print("Or follow the pipeline in dlc_training.ipynb to train our own model, then update config_dir and run this step again.")

### Step 7:  Ensemble
The output of the pipeline is the coordinates of snout and tailbase of two mice which are saved as features_mouse1_ensemble.csv and features_mouse2_ensemble.csv

In [ ]:
features_mouse1_ensemble = ensemble_features(features_mouse1_md, features_mouse1_dlc, tracking_dir, mouse_id=1)    
features_mouse2_ensemble = ensemble_features(features_mouse2_md, features_mouse2_dlc, tracking_dir, mouse_id=2) 

print("Pls find the output results in the directory {}".format(os.path.dirname(frames_dir)))